In [16]:
import numpy
import urllib
import scipy.optimize
import random
import csv
from math import exp
from math import log

print "Reading data..."
with open('winequality-white.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    data = []
    for row in reader:
        data.append(row)
title = data[0]
half = (len(data)-1)/2
trainData = data[1:half+1]
testData = data[half+1:]
print "done"

Reading data...
done


In [53]:
def featureX(datum):
    feat = [1] + [float(datum[i]) for i in range(11)]
    return feat

def featureY(datum):
    if float(datum[11]) <= 5:
        return 0
    else:
        return 1
    
X_train = [featureX(d) for d in trainData]
y_train = [featureY(d) for d in trainData]

X_test = [featureX(d) for d in testData]
y_test = [featureY(d) for d in testData]

In [74]:
def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        loglikelihood -= log(1 + exp(-logit))
        if not y[i]:
            loglikelihood -= logit
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
    #print "ll =", loglikelihood
    return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0.0]*len(theta)
    for i in range(len(X)):
        # Fill in code for the derivative
        logit = inner(X[i], theta)
        for j in range(len(theta)):
            dl[j] += (y[i] - sigmoid(logit)) * X[i][j]
    for j in range(len(theta)):
        dl[j] -=  2 * lam * theta[j]
    # Negate the return value since we're doing gradient *ascent*
    return numpy.array([-x for x in dl])

In [73]:
def accuracy(theta, X, y):
    correct = 0
    for i in range(len(y)):
        if sigmoid(inner(X[i], theta)) > 0.5:
            predict = 1
        else:
            predict = 0
        if predict == y[i]:
            correct += 1
    return  float(correct) / len(y)

In [77]:
theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X_train[0]), fprime, args = (X_train, y_train, 1.0))
print "Final log likelihood =", -l

Final log likelihood = -1383.18364755


In [78]:
ac = accuracy(theta, X_test, y_test)
print "Accuracy = ", ac

Accuracy =  0.766843609637
